In [80]:
import casadi as ca
from casadi import sin, cos, arctan, exp
from matplotlib import pyplot as plt
import numpy as np
from ipywidgets import interact

In [81]:
# define states
states = ca.MX.sym('states',7)
x, y, v, beta, psi, wz, delta = states[0], states[1], states[2], states[3], states[4], states[5], states[6]

In [82]:
# define controls
controls = ca.MX.sym('controls', 4)
wdelta = controls[0]
brake_force = controls[1]
accelerator = controls[2]
gear = controls[3]

In [83]:
# parameters
m = 1239
g = 9.81
lf = 1.19016
lr = 1.37484
eSP = 0.5
R = 0.302
Izz = 1752
cw = 0.3
rho = 1.249512
A = 1.4378946874
ig1, ig2, ig3, ig4, ig5 = 3.91, 2.002, 1.33, 1.0, 0.805
it = 3.91
Bf, Br, Cf, Cr, Df, Dr, Ef, Er = 10.96, 12.67, 1.3, 1.3, 4560.40, 3947.81, -0.5, -0.5
fR0, fR1, fR4 = 0.009, 0.002, 0.0003
B = 1.5 # car width
# Aerodynamic forces
FAx = 0.5 * cw * rho * A * v**2 
FAy = 0.0
ig_mu = ca.if_else(gear<1.5, ig1,
            ca.if_else(gear<2.5, ig2,
            ca.if_else(gear<3.5, ig3,
            ca.if_else(gear<4.5, ig4,
                ig5))))

In [84]:
dpsidt = wz

# calculate slip angles
alphaf = delta - arctan((lf*dpsidt - v*sin(beta))/(v*cos(beta)))
alphar = arctan((lr*dpsidt + v*sin(beta))/(v*cos(beta)))
# calculate lateral tire forces wear
# Fsf depends on alphaf, Fsr depends on alphar
Fsf = Df * sin(Cf*arctan(Bf*alphaf - Ef*(Bf*alphaf - arctan(Bf*alphaf))))
Fsr = Dr * sin(Cr*arctan(Br*alphar - Er*(Br*alphar - arctan(Br*alphar))))

# static tire loads at the front and rear wheel
Fzf = m*lr*g / (lf+lr)
Fzr = m*lf*g / (lf+lr)

# front, rear breaking forces
FBf = 2/3*brake_force
FBr = 1/3*brake_force

# friction coefficient
fRv = fR0 + fR1 * v/100 + fR4 * (v/100)**4

# rolling resistance forces
FRf = fRv * Fzf
FRr = fRv * Fzr

# motor torque
wmot = v * ig_mu * it / R

f1 = 1 - exp(-3*accelerator)
f2 = -37.8 + 1.54*wmot - 0.0019*wmot**2
f3 = -34.9 - 0.04775*wmot 

M_mot = f1*f2 + (1.0 - f1)*f3 
M_wheel = ig_mu * it * M_mot
Flr = M_wheel / R - FBr - FRr
Flf = - FBf - FRf 

dxdt = v * cos(psi - beta)
dydt = v * sin(psi - beta)
dvdt = 1/m * ( (Flr - FAx)*cos(beta) + Flf*cos(delta+beta) - (Fsr - FAy)*sin(beta) - Fsf*sin(delta + beta) ) 
dbetadt = wz - 1/(m*v) * ( (Flr - FAx)*sin(beta) + Flf*sin(delta+beta) + (Fsr - FAy)*cos(beta) + Fsf*cos(delta + beta))
dwzdt = 1/Izz * (Fsf*lf*cos(delta) - Fsr*lr - FAy*eSP + Flf*lf*sin(delta))
ddeltadt = wdelta

car_dynamics = ca.vertcat(
    dxdt,
    dydt,
    dvdt,
    dbetadt,
    dpsidt,
    dwzdt,
    ddeltadt
)

ode = {'x': states, 'p': controls, 'ode': car_dynamics}

dt = 0.05  # time step [s] (tune as needed for the track length)
integrator = ca.integrator('integrator', 'cvodes', ode, 0, dt)

In [85]:
h1, h2, h3, h4 = 1.1*B + 0.25, 3.5, 1.2*B+3.75, 1.3*B + 0.25

def Pl(x):
    if x<=44 or x>71:
        return 0
    elif x>44 and x<=44.5:
        return 4*h2*(x-44)**3
    elif x>44.5 and x<=45:
        return 4*h2*(x-45)**3 + h2
    elif x>45 and x<=70:
        return h2
    elif x>70 and x<=70.5:
        return 4*h2*(70-x)**3 + h2
    elif x>70.5 and x<=71:
        return 4*h2*(71-x)**3

Pl = np.vectorize(Pl)

def Pu(x):
    if x<=15:
        return h1
    elif x>15 and x<=15.5:
        return 4*(h3-h1)*(x-15)**3+h1
    elif x>15.5 and x<=16:
        return 4*(h3-h1)*(x-16)**3+h3
    elif x>15 and x<=94:
        return h3
    elif x>94 and x<=94.5:
        return 4*(h3-h4)*(94-x)**3+h3
    elif x>94.5 and x<=95:
        return 4*(h3-h4)*(95-x)**3+h4
    if x>95:
        return h4
    
Pu = np.vectorize(Pu)

In [86]:
%matplotlib inline
@interact(acceleration=(0.0,1.0,0.1), steer_angle = (-0.05,0.05,0.001), breaking_force = (0,15000,100), gear = (1,5,1))
def plot_traj(acceleration=5.5, steer_angle = 0, breaking_force = 0, gear =1):
    x0 = ca.DM([
        0, # initial x
        1, # initial y
        10.0, # initial speed
        0.0, # beta
        0.0, # initial psi
        0.0, # initial wz
        0.0  # initial delta
    ])
    u = ca.DM([
        steer_angle,
        breaking_force,
        acceleration,
        gear
    ])

    T = 10.0
    dt = 0.05
    N = int(T / dt)
    x_border = np.linspace(0,110,111)
    trajectory = [x0]
    xk = x0

    for _ in range(N):
        res = integrator(x0=xk, p=u)
        xk = res['xf']
        trajectory.append(xk)
    trajectory = np.array([np.array(x).squeeze() for x in trajectory])
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(trajectory[:,0], trajectory[:,1], color='red')
    ax.plot(x_border, Pl(x_border), color = 'k')
    ax.plot(x_border, Pu(x_border), color = 'k')
    ax.set_ylim(-5,15)
    ax.set_xlim(-5,115)
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='acceleration', max=1.0), FloatSlider(value=0.0, desc…